In [2]:
import pandas as pd
import numpy as np

In [3]:
tags = ['ID', 'offset', 'bid_1', 'ask_1', 'bid_2', 'ask_2', 'bid_size_1', 'ask_size_1', 'bid_size_2', 'ask_size_2', 'bid_entry_1', 'ask_entry_1', 'bid_entry_2', 'ask_entry_2', 'bid_entropy_1', 'ask_entropy_1', 'bid_entropy_2', 'ask_entropy_2', 'bid_sqentry_1', 'ask_sqentry_1', 'bid_sqentry_2', 'ask_sqentry_2', 'nb_trade']

In [16]:
DATAPATH = '/media/dhruv/New Volume/data challenge/data/smaller files'

In [17]:
input_data_file = DATAPATH +'/xaa'

In [18]:
input_data = pd.read_csv(input_data_file)

In [20]:
unique_ids = input_data.ID.unique()

In [32]:
input_data_dict = {elem : pd.DataFrame for elem in unique_ids}

In [176]:
target_data_file = DATAPATH+'/xaa_target'

In [177]:
target_data = pd.read_csv(target_data_file, delimiter = ';')

In [45]:
target_data_dict = {elem: pd.DataFrame for elem in unique_ids}

In [166]:
for keys in target_data_dict.keys():
    target_data_dict[key] = target_data[:][target_data.ID == key]

In [50]:
len(target_data_dict) == len(input_data_dict)

True

Now that we have the individual data frames and the target data indexed by ids, lets make a list of the useful quantities that we would need to build the features. To this end, lets just take a particular data frame for the input_data_dict

In [191]:
test_df = input_data_dict[45]

Lets create a hash-table for local and global variables. The local variables are those defined for each offset whilst global variables are max-min, std and change variables. 

### Test run

In [192]:
test_dict = {}

In [193]:
test_dict['df'] = test_df

In [128]:
useful_tags = tags[2:]

In [129]:
test_dict['stats'] = test_dict['df'].describe()

In [130]:
y = test_dict['stats']

In [131]:
y.count()['bid_1']

8

In [223]:
def is_change(test_dict,tag):
    ## check whether the tag changed in the data frame 
    ## once checked create a new entry in the dictionary as a key-value pair
    ser = test_dict['df'][tag]
    #ser.index = range(len(ser))
    count = 0
    for i in range(1,len(ser)):
        if ser[i-1] != ser[i]:
            count +=1
    ## creating key-value pairs 
    test_dict['max_'+tag] = max(ser)
    test_dict['min_'+tag] = min(ser)
    test_dict['change_by_'+tag] = -ser[0]+ser[len(ser)-1]
    test_dict['if_change_'+tag] = (count !=0)
    test_dict['mean_'+tag] = ser.mean()
    test_dict['change_times_'+tag] = count
    test_dict['target'] = target_data.TARGET
    

In [143]:
for tag in useful_tags:
    is_change(test_dict,tag)

In [134]:
ser = test_dict['df'][useful_tags[2]]

In [141]:
ser.index = range(len(ser))
print ser.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')


In [174]:
test_dict['df'].index = range(len(test_dict['df']))
test_dict['df']

,ID,offset,bid_1,ask_1,bid_2,ask_2,bid_size_1,ask_size_1,bid_size_2,ask_size_2,...,ask_entry_2,bid_entropy_1,ask_entropy_1,bid_entropy_2,ask_entropy_2,bid_sqentry_1,ask_sqentry_1,bid_sqentry_2,ask_sqentry_2,nb_trade
0,100,-1000,1444.0,1445.0,1443.0,1446.0,164,260,535,569,...,73,1.515264,1.936144,2.332604,2.418653,1.0,49.0,1.0,1.0,0.0
1,100,-500,1444.0,1445.0,1443.0,1446.0,164,260,535,569,...,73,1.515264,1.936144,2.332604,2.418653,1.0,49.0,1.0,1.0,0.0
2,100,-200,1444.0,1445.0,1443.0,1446.0,167,223,535,560,...,71,1.580532,1.744947,2.332604,2.410262,4.0,49.0,1.0,1.0,0.0
3,100,-100,1444.0,1445.0,1443.0,1446.0,167,223,535,560,...,71,1.580532,1.744947,2.332604,2.410262,4.0,49.0,1.0,1.0,0.0
4,100,-50,1444.0,1445.0,1443.0,1446.0,167,223,535,560,...,71,1.580532,1.744947,2.332604,2.410262,4.0,49.0,1.0,1.0,0.0
5,100,-20,1444.0,1445.0,1443.0,1446.0,167,223,535,560,...,71,1.580532,1.744947,2.332604,2.410262,4.0,49.0,1.0,1.0,0.0
6,100,-10,1444.0,1445.0,1443.0,1446.0,167,223,535,560,...,71,1.580532,1.744947,2.332604,2.410262,4.0,49.0,1.0,1.0,0.0
7,100,0,1444.0,1445.0,1443.0,1446.0,167,223,535,560,...,71,1.580532,1.744947,2.332604,2.410262,4.0,49.0,1.0,1.0,0.0


### Get target value given an id

In [189]:
dummy_index = list(target_data.ID)
target_data.index = dummy_index
target_data.TARGET[2]

0

In [201]:
dummy_index_2 = len(test_dict['df'].ID)
test_dict['df'].index= list(range(dummy_index_2))
test_dict['df'].ID[0]

45

## Lets loop this!

We shall test this for a small number of ids say 100 or so. 

In [212]:
%%time
hash_list = {}
for unique_id in unique_ids[0:100]:
    test_dict = {}
    test_dict['df'] = input_data_dict[unique_id]
    test_dict['df'].index = list(range(len(test_dict['df'].ID)))
    test_dict['target'] = target_data.TARGET[unique_id]
    test_dict['stats'] = test_dict['df'].describe()
    for tag in useful_tags:
        is_change(test_dict,tag)
    hash_list[unique_id] = test_dict
    

CPU times: user 8.75 s, sys: 4.02 ms, total: 8.75 s
Wall time: 8.75 s


## Sanity Checks

In [214]:
check_df = hash_list[65]['df']
check_df


,ID,offset,bid_1,ask_1,bid_2,ask_2,bid_size_1,ask_size_1,bid_size_2,ask_size_2,...,ask_entry_2,bid_entropy_1,ask_entropy_1,bid_entropy_2,ask_entropy_2,bid_sqentry_1,ask_sqentry_1,bid_sqentry_2,ask_sqentry_2,nb_trade
0,65,-1000,1448.0,1449.0,1447.0,1450.0,92,287,548,903,...,74,1.098612,2.358446,2.405047,2.476868,3600.0,100.0,1.0,1.0,0.0
1,65,-500,1447.0,1448.0,1446.0,1449.0,524,60,622,387,...,62,2.383791,1.329661,2.241951,2.382440,100.0,36.0,9.0,25.0,7.0
2,65,-200,1447.0,1448.0,1446.0,1449.0,519,49,627,387,...,62,2.384969,1.332179,2.249656,2.382440,100.0,4.0,25.0,25.0,8.0
3,65,-100,1447.0,1448.0,1446.0,1449.0,519,47,627,387,...,62,2.384969,1.386294,2.249656,2.382440,100.0,900.0,25.0,25.0,8.0
4,65,-50,1447.0,1448.0,1446.0,1449.0,519,47,627,387,...,62,2.384969,1.386294,2.249656,2.382440,100.0,900.0,25.0,25.0,8.0
5,65,-20,1447.0,1448.0,1446.0,1449.0,519,47,627,387,...,62,2.384969,1.386294,2.249656,2.382440,100.0,900.0,25.0,25.0,8.0
6,65,-10,1447.0,1448.0,1446.0,1449.0,519,47,627,387,...,62,2.384969,1.386294,2.249656,2.382440,100.0,900.0,25.0,25.0,8.0
7,65,0,1447.0,1448.0,1446.0,1449.0,519,47,627,387,...,62,2.384969,1.386294,2.249656,2.382440,100.0,900.0,25.0,25.0,8.0


In [222]:
hash_list[65]['change_by_nb_trade']

-8.0

In [207]:
target_data.TARGET[65]

1

In [210]:
## final check 
for unique_id in unique_ids[0:100]:
    if hash_list[unique_id]['target'] != target_data.TARGET[unique_id]:
        print 'Mismatch'
    

## Hooray it works 

### Lets compute some useful statistics for these newly computed variables

In [233]:
hist_dict = {}
keys = hash_list[1].keys()

In [235]:
hist_dict = {key: hash_list[1][key] for key in keys if key not in ['target', 'df', 'stats']}

In [236]:
hist_dict

{'change_by_ask_1': 0.0,
 'change_by_ask_2': 0.0,
 'change_by_ask_entropy_1': 0.0,
 'change_by_ask_entropy_2': 0.0,
 'change_by_ask_entry_1': 0,
 'change_by_ask_entry_2': 0,
 'change_by_ask_size_1': 0,
 'change_by_ask_size_2': 0,
 'change_by_ask_sqentry_1': 0.0,
 'change_by_ask_sqentry_2': 0.0,
 'change_by_bid_1': 0.0,
 'change_by_bid_2': 0.0,
 'change_by_bid_entropy_1': 0.0,
 'change_by_bid_entropy_2': 0.0,
 'change_by_bid_entry_1': 0,
 'change_by_bid_entry_2': 0,
 'change_by_bid_size_1': 0,
 'change_by_bid_size_2': 0,
 'change_by_bid_sqentry_1': 0.0,
 'change_by_bid_sqentry_2': 0.0,
 'change_by_nb_trade': 0.0,
 'change_times_ask_1': 0,
 'change_times_ask_2': 0,
 'change_times_ask_entropy_1': 0,
 'change_times_ask_entropy_2': 0,
 'change_times_ask_entry_1': 0,
 'change_times_ask_entry_2': 0,
 'change_times_ask_size_1': 0,
 'change_times_ask_size_2': 0,
 'change_times_ask_sqentry_1': 0,
 'change_times_ask_sqentry_2': 0,
 'change_times_bid_1': 0,
 'change_times_bid_2': 0,
 'change_times